# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [1]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [3]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [4]:
earthquake_json['metadata']

{'generated': 1636072138000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2021-10-04&endtime=2021-11-03',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.12.3',
 'count': 9534}

Each element in the JSON array `features` is a row of data for our dataframe.

In [5]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [6]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.5,
  'place': '88 km SE of King Salmon, Alaska',
  'time': 1635897098098,
  'updated': 1635897381592,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak021e2gg6g4',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak021e2gg6g4&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 35,
  'net': 'ak',
  'code': '021e2gg6g4',
  'ids': ',ak021e2gg6g4,',
  'sources': ',ak,',
  'types': ',origin,',
  'nst': None,
  'dmin': None,
  'rms': 0.45,
  'gap': None,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.5 - 88 km SE of King Salmon, Alaska'},
 'geometry': {'type': 'Point', 'coordinates': [-155.7096, 58.0716, 117.1]},
 'id': 'ak021e2gg6g4'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [7]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.50,"88 km SE of King Salmon, Alaska",1635897098098,1635897381592,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak021e2gg6g4,",",ak,",",origin,",NaN,NaN,0.4500,NaN,ml,earthquake,"M 1.5 - 88 km SE of King Salmon, Alaska"
1,0.70,"34 km SE of Mina, Nevada",1635896352596,1635963537655,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nn00827325,",",nn,",",origin,phase-data,",15.0,0.03800,0.0994,93.03,ml,earthquake,"M 0.7 - 34 km SE of Mina, Nevada"
2,2.10,"4 km S of P?hala, Hawaii",1635896300200,1635896500250,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv72780612,",",hv,",",origin,phase-data,",36.0,NaN,0.1200,155.00,md,earthquake,"M 2.1 - 4 km S of P?hala, Hawaii"
3,1.23,"12 km E of Smithfield, Utah",1635896285940,1635945107460,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",uu60466517,",",uu,",",origin,phase-data,",17.0,0.07971,0.1400,87.00,ml,earthquake,"M 1.2 - 12 km E of Smithfield, Utah"
4,2.13,"16km ESE of Julian, CA",1635896203230,1635899763840,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,1.0,2.2,NaN,...,",ci39847383,",",ci,",",dyfi,focal-mechanism,nearby-cities,origin,pha...",77.0,0.04980,0.1800,44.00,ml,earthquake,"M 2.1 - 16km ESE of Julian, CA"


### (Optional) Write Data to CSV

In [7]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>